In [1]:
from sqlalchemy import TextClause, Row
import re

import pandas as pd
import sys


sys.path.insert(1, r"..\models")
sys.path.insert(2, r"..\etl")
sys.path.insert(3, r"..\predictions")
from src.models.lstm_model import load_model_weights_from_checkpoint
from connect_db import connect_to_grad_db
from preprocess import ready_text_for_pred


In [2]:

engine = connect_to_grad_db()

with engine.connect() as conn:
    query = TextClause("select * from trustpilot_api")
    content = conn.execute(query).fetchall()
  

In [4]:
  
df = pd.DataFrame(data=content)



,text,rating,title,creation_date,reviews_for_site,trustpilot_api_id,sentiment_pred
0,Never again. Have 2 Teslas. Rip off. \nUsed to...,1,Never again,2023-02-15 00:31:22+00:00,www.teslamotors.com,4,0.19307047128677368
1,"Just to balance things out a bit, I just had ...",5,Just to balance things out a bit,2023-01-27 16:17:22+00:00,www.teslamotors.com,15,3.778378486633301
2,"Love my new Model Y, it's definitely the best ...",5,Love my new Model Y,2023-02-17 04:03:26+00:00,www.teslamotors.com,21,4
3,Don't trust Tesla! Tesla pressured me to take ...,1,Don't trust Tesla,2023-02-06 16:17:28+00:00,www.teslamotors.com,151,1.424659252166748
4,"Love my new Model Y, it's definitely the best ...",5,Love my new Model Y,2023-02-17 04:03:26+00:00,www.teslamotors.com,41,4
...,...,...,...,...,...,...,...
1715,In regards to their solar panels for residenti...,1,Shoddy/non-existent service,2023-02-08 23:56:59+00:00,www.teslamotors.com,91,0.2566845715045929
1716,"Model 3 was delivered in Dec 2022, apart from ...",1,Model 3 was delivered in Dec 2022,2023-01-26 02:04:02+00:00,www.teslamotors.com,103,1.2491023540496826
1717,Appalling customer service and buying experien...,1,Appalling customer service and buying…,2023-02-16 23:11:03+00:00,www.teslamotors.com,106,0
1718,Excellent service\nWe have owned our model 3 f...,5,Excellent service,2023-02-05 00:14:35+00:00,www.teslamotors.com,115,2.8190460205078125


In [6]:
from tfidf_functions import *

df_hottest_word = hottest_word(df['text'])

df_concat = pd.concat([df['text'], df_hottest_word], axis=1)

df_concat

c:\Users\fehmm\OneDrive\Skrivebord\python\graduation_project\src\predictions\..\models\tfidf_functions.py:24: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 24 of the file c:\Users\fehmm\OneDrive\Skrivebord\python\graduation_project\src\predictions\..\models\tfidf_functions.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  review_text = BeautifulSoup(raw_review).get_text()


,text,hottest_word_1,hottest_word_2,hottest_word_3,hottest_word_4,hottest_word_5
0,Never again. Have 2 Teslas. Rip off. \nUsed to...,never,advocate,apparently,body,decency
1,"Just to balance things out a bit, I just had ...",beyond,man,mark,mobile,ranger
2,"Love my new Model Y, it's definitely the best ...",definitely,wheels,complaints,excellent,love
3,Don't trust Tesla! Tesla pressured me to take ...,tesla,gave,delivery,less,december
4,"Love my new Model Y, it's definitely the best ...",definitely,wheels,complaints,excellent,love
...,...,...,...,...,...,...
1715,In regards to their solar panels for residenti...,chat,left,support,trying,began
1716,"Model 3 was delivered in Dec 2022, apart from ...",much,bit,let,advance,aka
1717,Appalling customer service and buying experien...,times,agreed,collect,honour,compensation
1718,Excellent service\nWe have owned our model 3 f...,tyres,unlike,excellent,experienced,charge


In [4]:

df_clean = [ready_text_for_pred(text) for text in df['comment_text']]

# print(ready_text_for_pred(df['comment_text'][0]))

model = load_model_weights_from_checkpoint()



In [14]:
pred_text = model.predict(df_clean)

pred_scam = model.predict(["tesla is an obvious scam"])


1/1 [==============================] - 0s 60ms/step


In [15]:
pred_scam

array([[3.7527351]], dtype=float32)

In [8]:
stats = {sentence:pred for sentence, pred in zip(df['comment_text'], pred_text)}


In [10]:
df_results = pd.DataFrame(
    {
        "sentence":df['comment_text'],
        "prediction":pred_text.flatten()
    }
)

In [11]:
df_results

,sentence,prediction
0,I sold my TSLA shates after Musk bought Twitte...,3.500395
1,Bro what are you thinking naming your course h...,1.422423
2,Elon jumped the shark. He supports republicans...,3.442274
3,Tesla's DISASTER in China 🤐,2.338441
4,Tesla killed people in China and it’s going viral,2.016588
...,...,...
245,A wasteful and disoriented comparison among tw...,2.340192
246,Tesla will be a subsidiary of Ford. Maybe Ford...,2.092800
247,A week ago you had several videos about how aw...,2.498825
248,The amount of population that can afford them ...,2.092680
